# Download Data

In [1]:
!wget https://hkustconnect-my.sharepoint.com/:u:/g/personal/nnanda_connect_ust_hk/Ea1qSvDD0xlPoAMGLjCesaoBXRA1Ll71-dYKVd2xyYZuSw?download=1 -O "yelp_academic_dataset_review.json.zip"
!unzip "yelp_academic_dataset_review.json.zip"

--2021-04-05 05:34:05--  https://hkustconnect-my.sharepoint.com/:u:/g/personal/nnanda_connect_ust_hk/Ea1qSvDD0xlPoAMGLjCesaoBXRA1Ll71-dYKVd2xyYZuSw?download=1
Resolving hkustconnect-my.sharepoint.com (hkustconnect-my.sharepoint.com)... 13.107.136.9
Connecting to hkustconnect-my.sharepoint.com (hkustconnect-my.sharepoint.com)|13.107.136.9|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/nnanda_connect_ust_hk/Documents/COMP4332%20Project%20Data/yelp_academic_dataset_review.json.zip?originalPath=aHR0cHM6Ly9oa3VzdGNvbm5lY3QtbXkuc2hhcmVwb2ludC5jb20vOnU6L2cvcGVyc29uYWwvbm5hbmRhX2Nvbm5lY3RfdXN0X2hrL0VhMXFTdkREMHhsUG9BTUdMakNlc2FvQlhSQTFMbDcxLWRZS1ZkMnh5WVp1U3c_cnRpbWU9cmt1SWJmVDMyRWc [following]
--2021-04-05 05:34:05--  https://hkustconnect-my.sharepoint.com/personal/nnanda_connect_ust_hk/Documents/COMP4332%20Project%20Data/yelp_academic_dataset_review.json.zip?originalPath=aHR0cHM6Ly9oa3VzdGNvbm5lY3QtbXkuc2hhcmVwb2ludC5jb20vOnU6L2cvcGVyc29uYWwvbm5hbmRh

# Import Libraries

In [2]:
!pip -q install keras-layer-normalization

In [25]:
import os
import nltk
import math
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from collections import Counter
from itertools import chain
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, BatchNormalization,\
    Activation, Input, Add, Concatenate, Embedding, Conv1D, MaxPool1D,\
    Flatten, LSTM, Bidirectional, MaxPooling1D, SimpleRNN, GRU, SpatialDropout1D
from keras_layer_normalization import LayerNormalization
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import sklearn.metrics as metrics
from sklearn.metrics import classification_report, accuracy_score


In [4]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
stopwords = set(stopwords.words("english"))
ps = PorterStemmer()

# Load the Yelp data

In [6]:
import json
import pandas as pd

reviews = ''

line_count = 0
with open('yelp_academic_dataset_review.json','r') as f:
    for line in f:
        reviews += line
        line_count += 1
        if line_count >= 50000:
            break

test_df = pd.read_json(reviews,lines=True)
print(test_df)

                    review_id  ...                date
0      lWC-xP3rd6obsecCYsGZRg  ... 2014-10-11 03:34:02
1      8bFej1QE5LXp4O05qjGqXA  ... 2015-07-03 20:38:25
2      NDhkzczKjLshODbqDoNLSg  ... 2013-05-28 20:38:06
3      T5fAqjjFooT4V0OeZyuk1w  ... 2010-01-08 02:29:15
4      sjm_uUcQVxab_EeLCqsYLg  ... 2011-07-28 18:05:01
...                       ...  ...                 ...
49995  eLkFmRh40KVwtc93cVZDJw  ... 2017-01-29 00:01:36
49996  eNIn4FkfY_5zL9zW7aNs9w  ... 2018-07-18 02:18:19
49997  0gcsSmvDOHBBF2TNbTd1KA  ... 2011-03-22 19:34:01
49998  5cTLlmyxjPxPfrsaGF-eyA  ... 2018-04-16 03:08:45
49999  uP2lNP3autkVrxYD_fHXbQ  ... 2013-03-19 18:44:05

[50000 rows x 9 columns]


In [7]:
test_df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,lWC-xP3rd6obsecCYsGZRg,ak0TdVmGKo4pwqdJSTLwWw,buF9druCkbuXLX526sGELQ,4,3,1,1,Apparently Prides Osteria had a rough summer a...,2014-10-11 03:34:02
1,8bFej1QE5LXp4O05qjGqXA,YoVfDbnISlW0f7abNQACIg,RA4V8pr014UyUbDvI-LW2A,4,1,0,0,This store is pretty good. Not as great as Wal...,2015-07-03 20:38:25
2,NDhkzczKjLshODbqDoNLSg,eC5evKn1TWDyHCyQAwguUw,_sS2LBIGNT5NQb6PD1Vtjw,5,0,0,0,I called WVM on the recommendation of a couple...,2013-05-28 20:38:06
3,T5fAqjjFooT4V0OeZyuk1w,SFQ1jcnGguO0LYWnbbftAA,0AzLzHfOJgL7ROwhdww2ew,2,1,1,1,I've stayed at many Marriott and Renaissance M...,2010-01-08 02:29:15
4,sjm_uUcQVxab_EeLCqsYLg,0kA0PAJ8QFMeveQWHFqz2A,8zehGz9jnxPqXtOc7KaJxA,4,0,0,0,The food is always great here. The service fro...,2011-07-28 18:05:01


In [12]:
import random
random.seed(62)

shuffle_indexs = random.sample(range(50000), 2000)
#print(shuffle_indexs)
review_valid_df = test_df.loc[shuffle_indexs,:] 
print(review_valid_df)

                    review_id  ...                date
37700  8tjNd6LvluVhaipzgmX-ew  ... 2017-05-27 00:50:37
11339  vp5VAvkpQaooGUx85DukDA  ... 2015-08-30 16:26:32
4269   JY2J5DIoCQQF5kwRmDfoow  ... 2008-04-29 17:23:43
15604  Jshvim5vcTVEaRYpQDR-sg  ... 2018-05-08 19:17:54
30457  pdjCrO3U3OtuaantXFmpQA  ... 2014-09-11 04:24:21
...                       ...  ...                 ...
44786  2uxJn1SI60IPhNzyzHwJLw  ... 2008-05-06 13:06:51
31031  ybjDk1AV7E1yn2cc9k3Z5Q  ... 2005-08-01 03:31:49
16476  LGMt_1e1hyQ_oI9uZNnITQ  ... 2018-04-15 17:37:13
16193  OIM2emjfjg_vp9dK5DKizQ  ... 2013-10-24 17:12:31
18409  _UGinps-2O2ArIFhA2ACBQ  ... 2016-07-04 03:18:37

[2000 rows x 9 columns]


In [13]:
review_valid_df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
37700,8tjNd6LvluVhaipzgmX-ew,_32MTdWeuZR5cylyCjFWig,cWgni4c-EZhsbQBoCwANhw,5,0,0,0,Just enjoyed a great dinner. Sushi was super f...,2017-05-27 00:50:37
11339,vp5VAvkpQaooGUx85DukDA,JDSWvOD7-ijWynQ7JLO1fg,vKchvHvmBQn1BmZhhbY2cg,1,0,0,0,"Bad service, no matter what you want to buy, g...",2015-08-30 16:26:32
4269,JY2J5DIoCQQF5kwRmDfoow,SIP1Dfwy1M0HWzdgGg9H9g,WJ9efJqZotX4J_KOGNxrtw,3,2,3,2,You stop right in front of my house on weekday...,2008-04-29 17:23:43
15604,Jshvim5vcTVEaRYpQDR-sg,P403AMmWidji525WVKE30Q,zNeUKQpxP-IKpYYVLRKxgg,4,0,0,0,Went there the other day and ordered a couple ...,2018-05-08 19:17:54
30457,pdjCrO3U3OtuaantXFmpQA,_rvxykxj_yGsB_Mt9ugAUw,JTGjHr9ii1TWq0TNLFkiQw,4,2,0,0,Blossom Tree is a great asian option on Downto...,2014-09-11 04:24:21


# Load the best Bert model

In [14]:
!pip -q install transformers

     |████████████████████████████████| 2.0MB 20.9MB/s 
     |████████████████████████████████| 3.2MB 50.4MB/s 
     |████████████████████████████████| 870kB 43.6MB/s 


In [15]:
#######################################
### -------- Load libraries ------- ###
# Load Huggingface transformers
from transformers import TFBertModel,  BertConfig, BertTokenizerFast
# Then what you need from tensorflow.keras
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop, Nadam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
# And pandas for data import + sklearn because you allways need sklearn
import pandas as pd
from sklearn.model_selection import train_test_split

In [16]:
#######################################
### --------- Setup BERT ---------- ###
# Name of the BERT model to use
model_name = 'bert-base-uncased'
# Max length of tokens
max_length = 100
# Load transformers config and set output_hidden_states to False
config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False
# Load BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)

In [17]:
# Ready output data for the model
y_valid = to_categorical(review_valid_df["stars"]-1, num_classes=5)

x_valid_attention_mask = tokenizer(
    text=review_valid_df["text"].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [18]:
!wget -q https://hkustconnect-my.sharepoint.com/:u:/g/personal/nnanda_connect_ust_hk/EXrPZ5UNreNKsJhqoZgBbuABcVH8WpSlYWxgiNC23weYJg?download=1 -O "weights_bert_with_attention_mask_62_65.hdf5"

model = keras.models.load_model(os.path.join("weights_bert_with_attention_mask_62_65.hdf5"))

# train_score = model.evaluate(
#     x={'input_ids': x_train_attention_mask['input_ids'], 'attention_mask': x_train_attention_mask['attention_mask']},
#     y={'stars': y_train})

valid_score = model.evaluate(
    x={'input_ids': x_valid_attention_mask['input_ids'], 'attention_mask': x_valid_attention_mask['attention_mask']},
    y={'stars': y_valid})

# print("training loss:", train_score[0], "training accuracy", train_score[1])
print("valid loss:", valid_score[0], "valid accuracy", valid_score[1])

63/63 [==============================] - 18s 202ms/step - loss: 0.8993 - accuracy: 0.6310
valid loss: 0.8992549777030945 valid accuracy 0.6309999823570251


Test 50k

In [19]:
# Ready output data for the model
y_valid_50k = to_categorical(test_df["stars"]-1, num_classes=5)

x_valid_attention_mask_50k = tokenizer(
    text=test_df["text"].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [20]:
model = keras.models.load_model(os.path.join("weights_bert_with_attention_mask_62_65.hdf5"))

valid_score = model.evaluate(
    x={'input_ids': x_valid_attention_mask_50k['input_ids'], 'attention_mask': x_valid_attention_mask_50k['attention_mask']},
    y={'stars': y_valid_50k})

print("valid loss:", valid_score[0], "valid accuracy", valid_score[1])

1563/1563 [==============================] - 344s 218ms/step - loss: 0.9372 - accuracy: 0.6190
valid loss: 0.9371615648269653 valid accuracy 0.6189799904823303


In [26]:
y_pred = model.predict(x={'input_ids': x_valid_attention_mask['input_ids'], 'attention_mask': x_valid_attention_mask['attention_mask']}, batch_size=64, verbose=1)
print(y_pred)

y_test = review_valid_df["stars"]
y_pred = np.argmax(y_pred,axis=1)+1

matrix = metrics.confusion_matrix(y_test, y_pred)
print(matrix)

#print("Accuracy: ",round(metrics.accuracy_score(y_test,y_pred),3))

# print("F1: ",round(metrics.f1_score(y_test, y_pred),3))
print(classification_report(y_test, y_pred))

print('accuracy', np.mean(y_test == y_pred))

acc = accuracy_score(y_test, y_pred)
print("accuracy score :", acc)

32/32 [==============================] - 14s 426ms/step
[[-2.0784929  -1.9992015  -0.9327136   2.0616372   4.214913  ]
 [ 4.355194    0.7897698  -1.4550432  -2.2701366  -1.7209439 ]
 [-1.6617411   0.14523715  2.3860233   0.82880515 -1.6135528 ]
 ...
 [-2.4687026  -2.0681925  -0.7254912   2.459964    3.688167  ]
 [-3.1055756  -1.655037    1.46684     2.7623305   0.7445159 ]
 [-2.3813508  -1.9550703  -0.18991452  2.4448674   3.2964742 ]]
[[213  22   6   1   3]
 [ 49  44  52   4   2]
 [ 13  35 132  44   9]
 [ 13   4 111 272 143]
 [ 16   3  25 183 601]]
              precision    recall  f1-score   support

           1       0.70      0.87      0.78       245
           2       0.41      0.29      0.34       151
           3       0.40      0.57      0.47       233
           4       0.54      0.50      0.52       543
           5       0.79      0.73      0.76       828

    accuracy                           0.63      2000
   macro avg       0.57      0.59      0.57      2000
weighted a